In [1]:
import numpy as np
import cv2
import os

<h1 style="text-align: center;">
    Camera Calibration with OpenCV
</h1>

In [2]:
chessboard_size = (8,6) # Your chessboard size

# iterative termination criteria, maximum iterationm and epsilon
term_criteria = (cv2.TermCriteria_EPS+ cv2.TermCriteria_MAX_ITER, 30, 0.001)

In [3]:
# List to store object points and images point from all the images.
obj_points = list() # 3D points in heterogeneous Xi
img_points = list() # 2D points on image xi

# Defining the world coordinates for 3D points Xi
objp = np.zeros((1, chessboard_size[0] * chessboard_size[1], 3), np.float32)
objp[0,:,:2] = np.mgrid[0:chessboard_size[0], 0:chessboard_size[1]].T.reshape(-1, 2)
prev_img_shape = None

In [4]:
image_path = os.getcwd()+'/images/stereo/calibrationdata/'
images = list()
image_size = None
print(image_path)

# Preparing calibration files
for filename in os.listdir(image_path) :
    print(filename)
    img = cv2.imread(image_path+'{}'.format(filename))
    images.append(img)

c:\Users\teera\Desktop\image processing\Final/images/stereo/calibrationdata/
left-0000.png
left-0001.png
left-0002.png
left-0003.png
left-0004.png
left-0005.png
left-0006.png
left-0007.png
left-0008.png
left-0009.png
left-0010.png
left-0011.png
left-0012.png
left-0013.png
left-0014.png
left-0015.png
left-0016.png
left-0017.png
left-0018.png
left-0019.png
left-0020.png
left-0021.png
left-0022.png
left-0023.png
left-0024.png
left-0025.png
left-0026.png
left-0027.png
left-0028.png
left-0029.png
left-0030.png
left-0031.png
left-0032.png
left-0033.png
left-0034.png
left-0035.png
left-0036.png
left-0037.png
left-0038.png
left-0039.png
left-0040.png
left-0041.png
left-0042.png
left-0043.png
left-0044.png
left-0045.png
left-0046.png
left-0047.png
left-0048.png
left-0049.png
left-0050.png
left-0051.png
left-0052.png
left-0053.png
left-0054.png
left-0055.png
left-0056.png
right-0000.png
right-0001.png
right-0002.png
right-0003.png
right-0004.png
right-0005.png
right-0006.png
right-0007.png
right

<h2 style="text-align: center">
    <a href = "https://docs.opencv.org/4.5.3/d9/d0c/group__calib3d.html#ga93efa9b0aa890de240ca32b11253dd4a"> findChessboardCorners </a>
    is a built in function for detecting the chessboard corners <br>
    <a href = "https://docs.opencv.org/4.5.3/dd/d1a/group__imgproc__feature.html#ga354e0d7c86d0d9da75de9b9701a9a87e"> cornerSubPix </a>
    is a function that uses to refine the location of the corners <br>
    <a href = "https://docs.opencv.org/4.5.3/d9/d0c/group__calib3d.html#ga6a10b0bb120c4907e5eabbcd22319022"> drawChessboardCorners </a>
    is a utility function for draw the detected corner to an image
</h2>

In [5]:
for image in images :
    img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(img_gray, chessboard_size, None) # xi in pixel (20,15)
    image_size = img_gray.shape
    # If found add obj points, image points afterthat refining them
    if ret == True :
        #print('Found')
        obj_points.append(objp) # Add Xi 3D
        
        corners2 = cv2.cornerSubPix(img_gray, corners, (11,11), (-1,-1), term_criteria) # Refining xi -> xi in subpixel, xi -> floating point (19.7, 15.1)
        img_points.append(corners2) # Add xi 2D

        #Draw and display the chessboard corners
        img = cv2.drawChessboardCorners(image, chessboard_size, corners2, ret)
        cv2.imshow('frame', img)
        cv2.waitKey(100)

cv2.destroyAllWindows()

<h2 style="text-align: center">
    <a href = "https://docs.opencv.org/4.5.3/d9/d0c/group__calib3d.html#ga3207604e4b1a1758aa66acb6ed5aa65d"> calibrateCamera </a>
    is the main function which you can use to calibrate a pinhole camera model <br>
</h2>

In [6]:
# Start calibration argmin ||xi - PXi||^2 , xi = PXi
print('Calibration')
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(obj_points,img_points, image_size[::-1] , None, None)

Calibration


<h2 style="text-align: center">
    <a href = "https://docs.opencv.org/4.5.3/d9/d0c/group__calib3d.html#ga7a6c4e032c97f03ba747966e6ad862b1"> getOptimalNewCameraMatrix </a>
    The function computes and returns the optimal new camera intrinsic matrix based on the free scaling parameter <br>
</h2>

In [7]:
img = images[0].copy()
h, w = img.shape[:2]
newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),1,(w,h))
print(mtx)
print('Optimal K')
print(newcameramtx, roi)

[[726.47630219   0.         466.25758541]
 [  0.         732.0640605  251.997867  ]
 [  0.           0.           1.        ]]
Optimal K
[[728.84155273   0.         473.45067169]
 [  0.         728.33746338 251.73843936]
 [  0.           0.           1.        ]] (1, 6, 861, 468)


<h2 style="text-align: center">
    <a href = "https://docs.opencv.org/4.5.3/d9/d0c/group__calib3d.html#ga69f2545a8b62a6b0fc2ee060dc30559d"> undistort </a>
    The function transforms an image to compensate radial and tangential lens distortion. <br>
</h2>

In [8]:
dst = cv2.undistort(img, mtx, dist, None, newcameramtx)
undistored = cv2.undistort(img, mtx, dist, None, None)
x,y,w,h = roi
dst = dst[y:y+h, x:x+w]

In [9]:
mean_error = 0
for i in range(len(obj_points)) :
    reprojected_point,_ = cv2.projectPoints(obj_points[i], rvecs[i], tvecs[i], mtx, dist)
    error = cv2.norm(img_points[i], reprojected_point, cv2.NORM_L2)/len(reprojected_point)
    mean_error += error

print("Reprojection error: {}".format(mean_error))

while True :
    cv2.imshow('calibrated', dst)
    cv2.imshow('undistort', undistored)
    if cv2.waitKey(1) & 0xFF == 27 : 
        break
cv2.destroyAllWindows()

Reprojection error: 10.099807080135967


In [ ]:
#Saving parameters into numpy format
np.save("./camera_params/monocular_camera_params/ret", ret)
np.save("./camera_params/monocular_camera_params/K", mtx)
np.save("./camera_params/monocular_camera_params/dist", dist)
np.save("./camera_params/monocular_camera_params/rvecs", rvecs)
np.save("./camera_params/monocular_camera_params/tvecs", tvecs)